In [3]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.



In [4]:
!pip install PyAudio

In [5]:
import pyaudio
import os
import wave
import librosa
import numpy as np
import tensorflow as tf
import pandas as pd
import tqdm

In [6]:
from sys import byteorder
from array import array
from struct import pack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import train_test_split

Loading data

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [7]:
# Read the dataset from a CSV file
dataset = pd.read_csv("data.csv")

# Determine the number of total, male, and female samples in the dataset
total_samples = len(dataset)
male_count = len(dataset[dataset['gender'] == 1])
female_count = len(dataset[dataset['gender'] == 0])

# Display the counts
print("Total number of samples:", total_samples)
print("Number of male samples:", male_count)
print("Number of female samples:", female_count)

# Create an array to store 128 audio features for each sample
X = np.zeros((total_samples, 128))

# Create an array to store gender labels for each sample (1 for male, 0 for female)
y = np.zeros((total_samples, 1))

# Loop through each sample to extract features and labels
for i, (file_name, gender_label) in tqdm.tqdm(enumerate(zip(dataset['filename'], dataset['gender'])), "Loading data", total=total_samples):
    #feature_data = np.load("/content/gdrive/My Drive/" + file_name)
    feature_data = np.load(file_name)
    X[i] = feature_data
    y[i] = gender_label

# Preprocess the feature data for neural network input
X = tf.keras.preprocessing.sequence.pad_sequences(X , dtype=np.float32, maxlen=144, padding='post')
X = X.reshape(-1, 12, 12)
X = np.expand_dims(X, axis=3)

Total number of samples: 66938
Number of male samples: 33469
Number of female samples: 33469


Loading data: 100%|██████████| 66938/66938 [00:12<00:00, 5562.31it/s]


In [9]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

# Further split the training set into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=7)

# Flatten each sequence in the training features
X_flat = np.array([sequence.flatten() for sequence in X_train])

# Display the first 10 flattened sequences and their shape
#print(X_flat[:10])
#print(X_flat.shape)

# Convert flattened features to a DataFrame and drop the last 16 columns
# that we added to make our records 2D from 1D
X_flat = pd.DataFrame(X_flat)
X_flat = X_flat.iloc[:, :-16]
print(X_flat.shape)

# Train a logistic regression model on the processed training data
logreg = LogisticRegression(random_state=87, max_iter=100000).fit(X_flat, y_train.squeeze())

(48195, 128)


In [28]:
# Flatten each sequence in the test features
X_test_flat = np.array([sequence.flatten() for sequence in X_test])
X_test_flat = pd.DataFrame(X_test_flat)
X_test_flat = X_test_flat.iloc[:, :-16]

# Predict the labels for the test set
y_predtest = logreg.predict(X_test_flat)
print(y_predtest)
print(y_predtest.shape)

# Calculate the accuracy of the model on the test set
accuracy_test = accuracy_score(y_test, y_predtest)
print(accuracy_test)

# Display the accuracy of the model on both the training and test sets
print("Training set accuracy: {:.4f}".format(logreg.score(X_flat, y_train.squeeze())))
print("Test set accuracy: {:.4f}".format(logreg.score(X_test_flat, y_test)))

[1. 0. 0. ... 0. 0. 1.]
(13388,)
0.8001195100089632
Training set accuracy: 0.7945
Test set accuracy: 0.8001


**CNN 2D**

In [11]:
# Define the input layer with the specified shape and batch size
inputs = tf.keras.Input(shape=(X.shape[1], X.shape[2], X.shape[3]), batch_size = 64)

# Add convolutional and max pooling layers
x = tf.keras.layers.Conv2D(128, 3, activation='relu')(inputs)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(256, 2, activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(512, 1, activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)

# Flatten the output for the dense layer
x = tf.keras.layers.Flatten()(x)

# Add a dense layer
x = tf.keras.layers.Dense(1024, activation='relu')(x)

# Define the output layer with a sigmoid activation for binary classification
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# Create and compile the model
model = tf.keras.Model(inputs, outputs)
# using binary crossentropy as it's male/female classification (binary)
model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(64, 12, 12, 1)]         0         
                                                                 
 conv2d (Conv2D)             (64, 10, 10, 128)         1280      
                                                                 
 max_pooling2d (MaxPooling2  (64, 5, 5, 128)           0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (64, 4, 4, 256)           131328    
                                                                 
 max_pooling2d_1 (MaxPoolin  (64, 2, 2, 256)           0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (64, 2, 2, 512)           131584

In [12]:
batch_size = 64
epochs = 100

# Execute model training with training data and perform validation with the validation set
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_valid, y_valid))

# save the model to a file

#model.save("results/model.h5")  

# Assessing the model's performance on the test dataset
print(f"Evaluating the model using {len(X_test)} samples...")
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy*100:.2f}%")

Epoch 1/100
754/754 [==============================] - 8s 10ms/step - loss: 0.4470 - accuracy: 0.7946 - val_loss: 0.3743 - val_accuracy: 0.8465
Epoch 2/100
754/754 [==============================] - 8s 10ms/step - loss: 0.3263 - accuracy: 0.8629 - val_loss: 0.2879 - val_accuracy: 0.8840
Epoch 3/100
754/754 [==============================] - 7s 9ms/step - loss: 0.2856 - accuracy: 0.8833 - val_loss: 0.2673 - val_accuracy: 0.8990
Epoch 4/100
754/754 [==============================] - 7s 9ms/step - loss: 0.2614 - accuracy: 0.8939 - val_loss: 0.2459 - val_accuracy: 0.9014
Epoch 5/100
754/754 [==============================] - 7s 10ms/step - loss: 0.2426 - accuracy: 0.9029 - val_loss: 0.2467 - val_accuracy: 0.9014
Epoch 6/100
754/754 [==============================] - 8s 11ms/step - loss: 0.2248 - accuracy: 0.9095 - val_loss: 0.2220 - val_accuracy: 0.9148
Epoch 7/100
754/754 [==============================] - 7s 10ms/step - loss: 0.2103 - accuracy: 0.9170 - val_loss: 0.2176 - val_accuracy: 0

In [26]:
inputs = tf.keras.Input(shape=(X.shape[1], X.shape[2], X.shape[3]), batch_size = 64)

x2 = tf.keras.layers.Conv2D(64, 3, activation='relu')(inputs)
x2 = tf.keras.layers.MaxPooling2D()(x2)

x2 = tf.keras.layers.Conv2D(128, 2, activation='relu')(x2)
x2 = tf.keras.layers.MaxPooling2D()(x2)

x2 = tf.keras.layers.Conv2D(256, 1, activation='relu')(x2)
x2 = tf.keras.layers.MaxPooling2D()(x2)

x2 = tf.keras.layers.Flatten()(x2)

x2 = tf.keras.layers.Dense(512, activation='relu')(x2)

outputs2 = tf.keras.layers.Dense(1, activation='sigmoid')(x2)

model2 = tf.keras.Model(inputs, outputs2)
model2.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")

model2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(64, 12, 12, 1)]         0         
                                                                 
 conv2d_3 (Conv2D)           (64, 10, 10, 64)          640       
                                                                 
 max_pooling2d_3 (MaxPoolin  (64, 5, 5, 64)            0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (64, 4, 4, 128)           32896     
                                                                 
 max_pooling2d_4 (MaxPoolin  (64, 2, 2, 128)           0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (64, 2, 2, 256)           3302

In [27]:
batch_size = 64
epochs = 100

model2.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_valid, y_valid))


print(f"Evaluating the model using {len(X_test)} samples...")
loss, accuracy = model2.evaluate(X_test, y_test, verbose=0)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy*100:.2f}%")

Epoch 1/100
754/754 [==============================] - 3s 4ms/step - loss: 0.4571 - accuracy: 0.7870 - val_loss: 0.3424 - val_accuracy: 0.8571
Epoch 2/100
754/754 [==============================] - 3s 5ms/step - loss: 0.3350 - accuracy: 0.8580 - val_loss: 0.2937 - val_accuracy: 0.8829
Epoch 3/100
754/754 [==============================] - 4s 5ms/step - loss: 0.2965 - accuracy: 0.8777 - val_loss: 0.3594 - val_accuracy: 0.8525
Epoch 4/100
754/754 [==============================] - 3s 4ms/step - loss: 0.2788 - accuracy: 0.8860 - val_loss: 0.2715 - val_accuracy: 0.8835
Epoch 5/100
754/754 [==============================] - 3s 4ms/step - loss: 0.2532 - accuracy: 0.8965 - val_loss: 0.2526 - val_accuracy: 0.8967
Epoch 6/100
754/754 [==============================] - 3s 4ms/step - loss: 0.2390 - accuracy: 0.9036 - val_loss: 0.2247 - val_accuracy: 0.9092
Epoch 7/100
754/754 [==============================] - 3s 4ms/step - loss: 0.2271 - accuracy: 0.9078 - val_loss: 0.2654 - val_accuracy: 0.8952

In [32]:
inputs = tf.keras.Input(shape=(X.shape[1], X.shape[2], X.shape[3]), batch_size = 64)

#decreasing the number of layers
x3 = tf.keras.layers.Conv2D(64, 3, activation='relu')(inputs)
x3 = tf.keras.layers.MaxPooling2D()(x3)

x3 = tf.keras.layers.Conv2D(128, 2, activation='relu')(x3)
x3 = tf.keras.layers.MaxPooling2D()(x3)

x3 = tf.keras.layers.Flatten()(x3)

x3 = tf.keras.layers.Dense(256, activation='relu')(x3)

outputs3 = tf.keras.layers.Dense(1, activation='sigmoid')(x3)

model3 = tf.keras.Model(inputs, outputs3)
model3.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")

model3.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(64, 12, 12, 1)]         0         
                                                                 
 conv2d_5 (Conv2D)           (64, 10, 10, 64)          640       
                                                                 
 max_pooling2d_5 (MaxPoolin  (64, 5, 5, 64)            0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (64, 4, 4, 128)           32896     
                                                                 
 max_pooling2d_6 (MaxPoolin  (64, 2, 2, 128)           0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (64, 512)                 0   

In [33]:
batch_size = 64
epochs = 100


model3.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_valid, y_valid))


model3.save("results/model.h5")  


print(f"Evaluating the model using {len(X_test)} samples...")
loss, accuracy = model3.evaluate(X_test, y_test, verbose=0)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy*100:.2f}%")

Epoch 1/100
754/754 [==============================] - 3s 3ms/step - loss: 0.4467 - accuracy: 0.8033 - val_loss: 0.3124 - val_accuracy: 0.8721
Epoch 2/100
754/754 [==============================] - 2s 3ms/step - loss: 0.3135 - accuracy: 0.8683 - val_loss: 0.2716 - val_accuracy: 0.8913
Epoch 3/100
754/754 [==============================] - 2s 3ms/step - loss: 0.2734 - accuracy: 0.8880 - val_loss: 0.2325 - val_accuracy: 0.9115
Epoch 4/100
754/754 [==============================] - 2s 3ms/step - loss: 0.2481 - accuracy: 0.9005 - val_loss: 0.2160 - val_accuracy: 0.9163
Epoch 5/100
754/754 [==============================] - 3s 3ms/step - loss: 0.2316 - accuracy: 0.9083 - val_loss: 0.2115 - val_accuracy: 0.9169
Epoch 6/100
754/754 [==============================] - 2s 3ms/step - loss: 0.2144 - accuracy: 0.9161 - val_loss: 0.2021 - val_accuracy: 0.9214
Epoch 7/100
754/754 [==============================] - 2s 3ms/step - loss: 0.2053 - accuracy: 0.9201 - val_loss: 0.1966 - val_accuracy: 0.9206

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Loss: 0.6306
Accuracy: 92.67%


**Demo**

In [13]:
pip install wavio

Note: you may need to restart the kernel to use updated packages.


In [14]:
import sounddevice as sd
import wavio
import scipy.stats

In [15]:
import sounddevice as sd
import numpy as np
from scipy.io.wavfile import write

def list_devices():
    #List all available audio input and output devices.

    print(sd.query_devices())

def record_audio(duration=5, fs=44100, channels=1, filename='recorded_voice.wav'):
    """
    Record audio for a given duration, sample rate, and number of channels.
    
    :param duration: Duration in seconds
    :param fs: Sampling rate
    :param channels: Number of audio channels
    :return: NumPy array of recorded audio
    """
    try:
        print("Recording...")
        recording = sd.rec(int(duration * fs), samplerate=fs, channels=channels)
        sd.wait()  # Wait until recording is finished
        print("Recording complete")
        print("Recording finished. Saving file...")
        wavio.write(filename, recording, fs, sampwidth=2)
        return recording
    except Exception as e:
        print(f"An error occurred: {e}")

# List available devices
list_devices()

  0 Rakhat’s iPhone Microphone, Core Audio (1 in, 0 out)
> 1 MacBook Pro Microphone, Core Audio (1 in, 0 out)
< 2 MacBook Pro Speakers, Core Audio (0 in, 2 out)
  3 BoomAudio, Core Audio (6 in, 6 out)
  4 Microsoft Teams Audio, Core Audio (2 in, 2 out)


In [16]:
def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    X, sample_rate = librosa.core.load(file_name)
    if chroma or contrast:
        stft = np.abs(librosa.stft(X))
    result = np.array([])
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, chroma))
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
        result = np.hstack((result, mel))
    if contrast:
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, contrast))
    if tonnetz:
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
        result = np.hstack((result, tonnetz))
    #result = np.hstack((result, np.zeros(1, 16)))
    for i in range(16):
        result = np.hstack((result, 0))
    #print(result.shape)
    #print(result)
    return result

In [17]:
pip install ffmpeg

Note: you may need to restart the kernel to use updated packages.


In [52]:

audio_data = record_audio(duration=5, fs=44100, channels=1)
features = extract_feature('recorded_voice.wav', mel=True)

features_logreg = pd.DataFrame(features)
features_logreg = features_logreg.iloc[:-16, :]

print()
print()

res_logreg = logreg.predict(features_logreg.T)[0]
female_logreg = 1 - res_logreg
gender_logreg = "male" if res_logreg > female_logreg else "female"
print("Result of LogReg demo:", gender_logreg)
print(f"Probabilities LR demo:     Male: {res_logreg*100:.2f}%    Female: {female_logreg*100:.2f}%")

print()
print()
features = features.reshape(-1, 12, 12)
#print(features)
features = np.expand_dims(features, axis=3)
#print(model.predict(features))
male_prob = model3.predict(features)[0][0]
female_prob = 1 - male_prob
gender = "male" if male_prob > female_prob else "female"
# show the result!
print("Result CNN demo:", gender)
print(f"Probabilities CNN demo:     Male: {male_prob*100:.2f}%    Female: {female_prob*100:.2f}%")

Recording...
Recording complete
Recording finished. Saving file...


Result of LogReg demo: female
Probabilities LR demo:     Male: 0.00%    Female: 100.00%


1/1 [==============================] - 0s 28ms/step
Result CNN demo: female
Probabilities CNN demo:     Male: 6.96%    Female: 93.04%
